# Regresiones lineales simples

El propósito de este script es determinar si un modelo lineal es adecuado para construir una función analítica entre el salario y las medidas del esfuerzo. Esto dependerá de la significancia estadística de las variables, así como el número que haya de estas en cada modelo.

In [1]:
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import pandas as pd
import numpy as np
import math
import os
import warnings
print('Modulos importados')

Modulos importados


In [2]:
# Configuraciones
warnings.filterwarnings('ignore')
# Reduzcamos el número de línea a leer
pd.options.display.max_rows = 5

In [11]:
# Veamos el directorio actual de trabajo
print(os.getcwd())
# El directorio anterior es el correcto, pero si no lo fuese, hacemos lo sigueinte:
path = '/home/usuario/Documentos/Github/Proyectos/MLB_HN'
os.chdir(path)

/home/usuario/Documentos/Github/Proyectos/MLB_HN


In [3]:
# Años de análisis
period = 11
csv = '.csv'
# Directorios
pitcher_path = '../../Data/New_Data/Free_Agent/Pitchers/complete_free_agents_pitchers_'
hitter_path = '../../Data/New_Data/Free_Agent/Hitters/complete_free_agents_hitters_'
# Originales:
df_pitcher = [None]*period
df_hitter = [None]*period

In [4]:
# Función para normalizar datos
def normalizar(df):
    df_normalizado = df.copy()
    for etiqueta in df.columns:
        maximo = df[etiqueta].max()
        minimo = df[etiqueta].min()
        df_normalizado[etiqueta] = (df[etiqueta] - minimo) / (maximo - minimo)
        
    return df_normalizado

In [5]:
for i in range(0,period):
    df_pitcher[i] = pd.read_csv(pitcher_path + str(2011 + i) + csv)
    df_hitter[i] = pd.read_csv(hitter_path + str(2011 + i) + csv)    

Veamos el tipo de datos contenidos en las bases de datos

In [6]:
df_pitcher[2].head()

,Jugador,Valor_contrato,Valor_promedio_contrato,Posicion,Juegos,Juegos_iniciados,Inning_pitched,Bateos,Carreras,Carreras_ganadas,...,Wins,Losses,Saves,WHIP,ERA,Equipo,Sueldo,Victorias,Juegos totales,Promedio_victorias
0,Zack Greinke,147000000,24500000,SP,29,28,177.7,152,54,52,...,15,4,0,1.11,2.63,LAD,19000000,92,162,0.567901
1,J.P. Howell,2850000,2850000,RP,67,0,62.0,42,14,14,...,4,1,0,1.05,2.03,LAD,2850000,92,162,0.567901
2,Brian Wilson,1000000,1000000,RP,18,0,13.7,8,1,1,...,2,1,0,0.88,0.66,LAD,1000000,92,162,0.567901
3,Peter Moylan,0,0,RP,14,0,15.3,23,11,11,...,1,0,0,1.96,6.46,LAD,490000,92,162,0.567901
4,Anibal Sanchez,80000000,16000000,RP,29,29,182.0,156,56,52,...,14,8,0,1.15,2.57,DET,8800000,93,162,0.574074


In [7]:
df_hitter[8].head()

,Jugador,Valor_contrato,Valor_promedio_contrato,Posicion,Juegos,Porcetnaje_juegos,At-bats,Bateos,Home-runs,RBI,Porcentaje_bateo,OPS,Equipo,Sueldo,Victorias,Juegos totales,Promedio_victorias
0,Bryce Harper,330000000,25384615,RF,157,0.969,573,149,35,114,0.260,0.882,PHI,11538462,81,162,0.500000
1,Andrew McCutchen,50000000,16666667,RF,59,0.364,219,56,10,29,0.256,0.834,PHI,10000000,81,162,0.500000
2,Blake Parker,0,0,RP,60,0.370,1,0,0,0,0.000,0.000,PHI,555000,81,162,0.500000
3,Manny Machado,300000000,30000000,3B,156,0.963,587,150,32,85,0.256,0.796,SD,12000000,70,162,0.432099
4,Garrett Richards,15500000,7750000,SP,3,0.019,2,0,0,0,0.000,0.000,SD,7000000,70,162,0.432099


Debido a la enorme diferencias en la magnitud entre el promedio de victorias y los sueldos, se tendrá que estadandarizar los sueldos. A pesar de que intuitivamente no tenemos que usar la columna del valor de contrato, también estandarizémosla.

In [9]:
# Originales:
df_pitcher_copy = [None]*period
df_hitter_copy = [None]*period

In [12]:
for i in range(1,period):
    df_pitcher_copy[i] = df_pitcher[i].copy()
    df_hitter_copy[i] = df_hitter[i].copy()
    
    maximo_p_1 = df_pitcher_copy[i]['Sueldo'].max()
    minimo_p_1 = df_pitcher_copy[i]['Sueldo'].min()
    
    maximo_h_1 = df_hitter_copy[i]['Sueldo'].max()
    minimo_h_1 = df_hitter_copy[i]['Sueldo'].min()
    
    df_pitcher_copy[i]['Sueldo_estandarizado'] = (df_pitcher_copy[i]['Sueldo'] - minimo_p_1)/(maximo_p_1 - minimo_p_1)
    df_hitter_copy[i]['Sueldo_estandarizado'] = (df_hitter_copy[i]['Sueldo'] - minimo_h_1)/(maximo_h_1 - minimo_h_1)
    
    maximo_p_2 = df_pitcher_copy[i]['Valor_contrato'].max()
    minimo_p_2 = df_pitcher_copy[i]['Valor_contrato'].min()
    
    maximo_h_2 = df_hitter_copy[i]['Valor_contrato'].max()
    minimo_h_2 = df_hitter_copy[i]['Valor_contrato'].min()
    
    df_pitcher_copy[i]['Valor_contrato_estandarizado'] = (df_pitcher_copy[i]['Valor_contrato'] - minimo_p_2)/(maximo_p_2 - minimo_p_2)
    df_hitter_copy[i]['Valor_contrato_estandarizado'] = (df_hitter_copy[i]['Valor_contrato'] - minimo_h_2)/(maximo_h_2 - minimo_h_2)

In [13]:
df_hitter_copy[3].head()

,Jugador,Valor_contrato,Valor_promedio_contrato,Posicion,Juegos,Porcetnaje_juegos,At-bats,Bateos,Home-runs,RBI,Porcentaje_bateo,OPS,Equipo,Sueldo,Victorias,Juegos totales,Promedio_victorias,Sueldo_estandarizado,Valor_contrato_estandarizado
0,Robinson Cano,240000000,24000000,2B,157,0.969,595,187,14,82,0.314,0.836,SEA,24050000,86,161,0.534161,1.000000,1.000000
1,Kendrys Morales,7540983,7540983,DH,98,0.605,367,80,8,42,0.218,0.612,SEA,12000000,86,161,0.534161,0.495446,0.031421
2,Chris Young,7250000,7250000,RF,111,0.685,325,72,11,38,0.222,0.710,SEA,4725000,86,161,0.534161,0.190829,0.030208
3,Chris Young,7250000,7250000,RP,30,0.185,1,0,0,0,0.000,0.000,SEA,4725000,86,161,0.534161,0.190829,0.030208
4,Corey Hart,6000000,6000000,RF,68,0.420,232,47,6,21,0.203,0.590,SEA,6000000,86,161,0.534161,0.244215,0.025000


Ahora, tanto la variable dependiente como independiente están en el intervalo $[0,1]$

In [14]:
df_pitcher_copy[2].head()

,Jugador,Valor_contrato,Valor_promedio_contrato,Posicion,Juegos,Juegos_iniciados,Inning_pitched,Bateos,Carreras,Carreras_ganadas,...,Saves,WHIP,ERA,Equipo,Sueldo,Victorias,Juegos totales,Promedio_victorias,Sueldo_estandarizado,Valor_contrato_estandarizado
0,Zack Greinke,147000000,24500000,SP,29,28,177.7,152,54,52,...,0,1.11,2.63,LAD,19000000,92,162,0.567901,1.000000,1.000000
1,J.P. Howell,2850000,2850000,RP,67,0,62.0,42,14,14,...,0,1.05,2.03,LAD,2850000,92,162,0.567901,0.127499,0.019388
2,Brian Wilson,1000000,1000000,RP,18,0,13.7,8,1,1,...,0,0.88,0.66,LAD,1000000,92,162,0.567901,0.027553,0.006803
3,Peter Moylan,0,0,RP,14,0,15.3,23,11,11,...,0,1.96,6.46,LAD,490000,92,162,0.567901,0.000000,0.000000
4,Anibal Sanchez,80000000,16000000,RP,29,29,182.0,156,56,52,...,0,1.15,2.57,DET,8800000,93,162,0.574074,0.448947,0.544218


## I. Victorias en función del salario

$$
\delta_{t} = a + b\omega_{t} + e_{t} 
$$
donde $\delta_{t}$ son las victorias por temporada. Se aplicará el modelo tanto para agentes libres que sean *hitters* o *pitchers*. Para cada año de análisis, consideraremos el promedio de victorias como medida del esfuerzo y el total del salario, para después hacer con el valor del contrato.

Creemos una lista para guardar los modelos y proceder a obtenerlos

In [15]:
# Modelos
hitter_model_I_a = [None]*period
pitcher_model_I_a = [None]*period

In [16]:
for i in range(1,period):
    # Hitters:
    y_hitter = df_hitter_copy[i]['Promedio_victorias'].tolist()
    x_hitter = df_hitter_copy[i]['Sueldo_estandarizado'].tolist()
    
    x_hitter = sm.add_constant(x_hitter)
    
    hitter_model_I_a[i] = sm.OLS(y_hitter, x_hitter).fit()
    
    # Pitchers:
    y_pitcher = df_pitcher_copy[i]['Promedio_victorias'].tolist()
    x_pitcher = df_pitcher_copy[i]['Sueldo_estandarizado'].tolist()
    
    x_pitcher = sm.add_constant(x_pitcher)
    
    pitcher_model_I_a[i] = sm.OLS(y_pitcher, x_pitcher).fit()

Observemos los resúmenes de las regresiones

#### Hitters

In [17]:
for i in range(1,period):
    print(hitter_model_I_a[i].summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.021
Method:                 Least Squares   F-statistic:                  0.001698
Date:                Wed, 29 Jun 2022   Prob (F-statistic):              0.967
Time:                        09:46:46   Log-Likelihood:                 65.615
No. Observations:                  49   AIC:                            -127.2
Df Residuals:                      47   BIC:                            -123.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5191      0.013     41.275      0.0

Los resultados muestran que, en general, hay poca significancia estadística para explicar las victorias con el salario de los agentes libres. Así mismo, tampoco tiene poder predictivo puesto que la $R^2$ es baja. Faltará corroborar esto con validación cruzada, sin embargo, el hecho de la baja significancia estadística no cambiará.

#### Pitchers

In [44]:
for i in range(1,period):
    print(pitcher_model_I_a[i].summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.051
Method:                 Least Squares   F-statistic:                    0.1269
Date:                Wed, 29 Jun 2022   Prob (F-statistic):              0.726
Time:                        01:05:24   Log-Likelihood:                 25.191
No. Observations:                  19   AIC:                            -46.38
Df Residuals:                      17   BIC:                            -44.49
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5194      0.027     19.110      0.0

Observamos los mismos problemas.

De esto, podemos concluir que este modelo no es el más adecuado, por lo que ahora se intentará con el resto de medidas de esfuerzo respectivamente para *hitters* y *pitchers*.

### Contrato

Análogamente, veamoslo para el valor del contrato del agente libre:

In [18]:
# Modelos
hitter_model_I_b = [None]*period
pitcher_model_I_b = [None]*period

In [72]:
for i in range(1,period):
    # Hitters:
    y_hitter = df_hitter_copy[i]['Promedio_victorias'].tolist()
    x_hitter = df_hitter_copy[i]['Valor_contrato_estandarizado'].tolist()
    
    x_hitter = sm.add_constant(x_hitter)
    
    hitter_model_I_b[i] = sm.OLS(y_hitter, x_hitter).fit()
    
    # Pitchers:
    y_pitcher = df_pitcher_copy[i]['Promedio_victorias'].tolist()
    x_pitcher = df_pitcher_copy[i]['Valor_contrato_estandarizado'].tolist()
    
    x_pitcher = sm.add_constant(x_pitcher)
    
    pitcher_model_I_b[i] = sm.OLS(y_pitcher, x_pitcher).fit()

#### Pitchers

In [73]:
for i in range(1,period):
    print(pitcher_model_I_b[i].summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     1.235
Date:                Wed, 29 Jun 2022   Prob (F-statistic):              0.282
Time:                        11:39:21   Log-Likelihood:                 25.786
No. Observations:                  19   AIC:                            -47.57
Df Residuals:                      17   BIC:                            -45.68
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5390      0.018     29.302      0.0

#### Hitters

In [74]:
for i in range(1,period):
    print(hitter_model_I_b[i].summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.020
Method:                 Least Squares   F-statistic:                   0.05244
Date:                Wed, 29 Jun 2022   Prob (F-statistic):              0.820
Time:                        11:39:40   Log-Likelihood:                 65.642
No. Observations:                  49   AIC:                            -127.3
Df Residuals:                      47   BIC:                            -123.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5197      0.010     51.845      0.0

A diferencia del salario, el valor del contrato tiende a tener un mayor grado de significancia estadística que en el caso del salario, aunque no el suficiente a lo largo de todos los años como para que se considere un modelo adecuado.

## II. Salario en función de las medidas de esfuerzo

$$
\omega_{t} = a + b\delta_{t} + e_{t} 
$$
donde $\delta_{t}$ es un vector que contiene todas las estadísticas deportivas. Se aplicará el modelo tanto para agentes libres que sean *hitters* o *pitchers*. Lo que se hará es regresar sobre todas las medidas del esfuerzo con el objetivo de determinar cuáles variables son significativas para el modelo y cuáles no

In [51]:
df_hitter_copy[1].columns

Index(['Jugador', 'Valor_contrato', 'Valor_promedio_contrato', 'Posicion',
       'Juegos', 'Porcetnaje_juegos', 'At-bats', 'Bateos', 'Home-runs', 'RBI',
       'Porcentaje_bateo', 'OPS', 'Equipo', 'Sueldo', 'Victorias',
       'Juegos totales', 'Promedio_victorias', 'Sueldo_estandarizado',
       'Valor_contrato_estandarizado'],
      dtype='object')

In [50]:
df_pitcher_copy[1].columns

Index(['Jugador', 'Valor_contrato', 'Valor_promedio_contrato', 'Posicion',
       'Juegos', 'Juegos_iniciados', 'Inning_pitched', 'Bateos', 'Carreras',
       'Carreras_ganadas', 'Walks', 'Strike-outs', 'Wins', 'Losses', 'Saves',
       'WHIP', 'ERA', 'Equipo', 'Sueldo', 'Victorias', 'Juegos totales',
       'Promedio_victorias', 'Sueldo_estandarizado',
       'Valor_contrato_estandarizado'],
      dtype='object')

In [69]:
for i in range(1,period):
    # Hitters:
    scaler_h = StandardScaler()
    x_hitter = df_hitter_copy[i].iloc[:, [4,5,6,7,8,9,10,11,14,15,16]]
    scaler_h.fit(x_hitter)
    scaler_h.transform(x_hitter)
    y_hitter = df_hitter_copy[i]['Sueldo_estandarizado']
    
    x_hitter = sm.add_constant(x_hitter)
    
    hitter_model_II_a[i] = sm.OLS(y_hitter, x_hitter).fit()
    
    # Pitchers:
    scaler_p = StandardScaler()
    x_pitcher = df_pitcher_copy[i].iloc[:, [4,5,6,7,8,9,10,11,12,13,14,15,16,19,20,21]]
    scaler_p.fit(x_pitcher)
    scaler_p.transform(x_pitcher)
    y_pitcher = df_pitcher_copy[i]['Sueldo_estandarizado']
    
    x_pitcher = sm.add_constant(x_pitcher)
    
    pitcher_model_II_a[i] = sm.OLS(y_pitcher, x_pitcher).fit()

In [68]:
# Modelos
hitter_model_II_a = [None]*period
pitcher_model_II_a = [None]*period

Lo siguiente es un código sin normalizar:

for i in range(1,period):
    # Hitters:
    x_hitter = df_hitter_copy[i][['Juegos', 'Porcetnaje_juegos', 'At-bats', 'Bateos', 'Home-runs', 'RBI',
                                  'Porcentaje_bateo', 'OPS', 'Victorias', 'Juegos totales', 
                                  'Promedio_victorias']]
    #x_hitter = normalizar(x_hitter)
    y_hitter = df_hitter_copy[i]['Sueldo_estandarizado']
    
    x_hitter = sm.add_constant(x_hitter)
    
    hitter_model_II_a[i] = sm.OLS(y_hitter, x_hitter).fit()
    
    # Pitchers:
    x_pitcher = df_pitcher_copy[i][['Juegos', 'Juegos_iniciados', 'Inning_pitched', 'Bateos', 'Carreras',
                                    'Carreras_ganadas', 'Walks', 'Strike-outs', 'Wins', 'Losses', 'Saves',
                                    'WHIP', 'ERA','Victorias', 'Juegos totales', 'Promedio_victorias']]
    #x_pitcher = normalizar(x_pitcher)
    y_pitcher = df_pitcher_copy[i]['Sueldo_estandarizado']
    
    x_pitcher = sm.add_constant(x_pitcher)
    
    pitcher_model_II_a[i] = sm.OLS(y_pitcher, x_pitcher).fit()

#### Hitters

In [70]:
for i in range(1,period):
    print(hitter_model_II_a[i].summary())

                             OLS Regression Results                             
Dep. Variable:     Sueldo_estandarizado   R-squared:                       0.322
Model:                              OLS   Adj. R-squared:                  0.166
Method:                   Least Squares   F-statistic:                     2.060
Date:                  Wed, 29 Jun 2022   Prob (F-statistic):             0.0580
Time:                          11:03:20   Log-Likelihood:                 19.582
No. Observations:                    49   AIC:                            -19.16
Df Residuals:                        39   BIC:                           -0.2454
Df Model:                             9                                         
Covariance Type:              nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Juegos          

Se aprecia que la valriable que tiende a ser estadísticamente significativa a lo largo de los años es *at-bats* y en menos ocasiones, secundándole, es *home-runs* y *victorias*.

#### Pitcher

In [71]:
for i in range(1,period):
    print(pitcher_model_II_a[i].summary())

                             OLS Regression Results                             
Dep. Variable:     Sueldo_estandarizado   R-squared:                       0.930
Model:                              OLS   Adj. R-squared:                  0.685
Method:                   Least Squares   F-statistic:                     3.797
Date:                  Wed, 29 Jun 2022   Prob (F-statistic):              0.103
Time:                          11:30:45   Log-Likelihood:                 18.467
No. Observations:                    19   AIC:                            -6.934
Df Residuals:                         4   BIC:                             7.233
Df Model:                            14                                         
Covariance Type:              nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Juegos          

Notamos que no hay alguna variable que persista más de tres años siendo estadísticamente significativa. EL comportamiento tiende a ser aleatorio.